In [1]:
import pandas
import numpy as np
import tensorflow as tf
from keras.utils import np_utils
# CNN related
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.recurrent import GRU
#from keras.layers.recurrent import LSTM, GRU
from keras.layers import Conv1D, MaxPooling1D, AtrousConvolution1D, RepeatVector
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger
from keras.layers.wrappers import Bidirectional
from keras import regularizers
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import *
from keras.optimizers import RMSprop, Adam, SGD, Nadam
from keras.initializers import *
import matplotlib.pyplot as plt

from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE # doctest: +NORMALIZE_WHITESPACE
from imblearn.over_sampling import RandomOverSampler # doctest: +NORMALIZE_WHITESPACE
from imblearn.over_sampling import SVMSMOTE
from imblearn.over_sampling import KMeansSMOTE
from imblearn.over_sampling import BorderlineSMOTE # doctest: +NORMALIZE_WHITESPACE
from imblearn.combine import SMOTETomek # doctest: +NORMALIZE_WHITESPACE

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from keras import backend as K


##################
# AUC for a binary classifier
def auc(y_true, y_pred):
    ptas = tf.stack([binary_PTA(y_true,y_pred,k) for k in np.linspace(0, 1, 1000)],axis=0)
    pfas = tf.stack([binary_PFA(y_true,y_pred,k) for k in np.linspace(0, 1, 1000)],axis=0)
    pfas = tf.concat([tf.ones((1,)) ,pfas],axis=0)
    binSizes = -(pfas[1:]-pfas[:-1])
    s = ptas*binSizes
    return K.sum(s, axis=0)
    
##################
# PFA, prob false alert for binary classifier
def binary_PFA(y_true, y_pred, threshold=K.variable(value=0.5)):
    y_pred = K.cast(y_pred >= threshold, 'float32')
    # N = total number of negative labels
    N = K.sum(1 - y_true)
    # FP = total number of false alerts, alerts from the negative class labels
    FP = K.sum(y_pred - y_pred * y_true)
    #print(FP/N)
    return FP/N

##################
# P_TA prob true alerts for binary classifier
def binary_PTA(y_true, y_pred, threshold=K.variable(value=0.5)):
    y_pred = K.cast(y_pred >= threshold, 'float32')
    # P = total number of positive labels
    P = K.sum(y_true)
    # TP = total number of correct alerts, alerts from the positive class labels
    TP = K.sum(y_pred * y_true)
    return TP/P

class ecgcnn: 
    np.set_printoptions(threshold=np.inf)
    def __init__(self, base_path, stkid, infosize, onehotsize ):
        self.stkid = stkid
        fname = base_path + stkid + '.csv'
        self.INFOSIZE = infosize
        self.ONEHOTSIZE = onehotsize
        self.load_data(fname)
        self.EMB_SIZE = self.INFOSIZE+self.ONEHOTSIZE
        #self.ONEHOTSIZE = onehotsize
        

##################
    def load_data(self, fname):
        # load csv data
        mat = pandas.read_csv(fname, sep=",", header=0, error_bad_lines=False).as_matrix()

        self.info = []
        self.sid=[]
        for i in range(self.INFOSIZE):
            self.info.append(self.matrix_col(mat, i))   
        self.sid.append(self.matrix_col(mat,9))
       # print(self.sid) 
        #x = self.find_category(self.sid)
        #print(x)
##################        
    '''
    def find_category0(self, sid):
        print(sid[0][0])
        y = list(map(int, sid[3:-1])) 
        #y = map(eval,sid[3:-1])
        print(y)    
        for n in range(len(sid)):
            sidnum = [0]*51 
            if ( y == 11 ):
                print("456") 
                sidnum [n]=sidnum [n]+1
                print(sidnum)
                return  sidnum
    '''
    def find_category(self, sid):
        result = []
        for sidstr in sid[0]:
            index = int(sidstr.replace('sid', ''))
            result.append(index)
            
        return result
        
    '''           
            sidnum = [0]*51
            #print(index)
            sidnum[index-1] = 1
            #print(sidnum)
            result.append(sidnum)
           
            return result
    ''' 
######################################################
    def matrix_col(self, matrix, i):
        return [row[i] for row in matrix]    
    
######################################################
    def normalize(self, data):
        #return (np.array(data)-np.mean(data))/np.std(data) 
        std = np.std(data)
        if (std == 0):
            data2 = np.zeros(len(data))
            return data2
        return (np.array(data)-np.mean(data))/std
    
######################################################
    
    def backtest_training_data(self,num,s):
        x_train0, y_train_onehot = [], []  
        x_train1, x_test1, y_train1, y_test1 = [], [], [], []
        # normalize
        nvals = []
        for cval in self.info:
            nvals.append(self.normalize(cval))
        
        # training preparation            
        l2t = nvals
        x_train0 = np.column_stack(l2t)
        y_train_onehot = self.find_category(self.sid)

        x_train = []
        for rec in x_train0:
            x_train.append(rec)
        
        x_train = np.array(x_train)
        #y_train_onehot = np_utils.to_categorical(y_train)
        y_train_onehot = np.array(y_train_onehot)

        if num==0:
            n = 40
        if num==1:
            n = 37
        if num==2:
            n = 16
        if num==3:
            n = 84
        x_train1, x_test1, y_train1, y_test1 = train_test_split(x_train, y_train_onehot, test_size=0.2, random_state=None)
        
        
        if s==0:
            x_res, y_res = x_train1, y_train1
        else:    
            if s==1:
                sm = SMOTE(random_state=0)
            if s==2:
                sm = SVMSMOTE(random_state=0)
            if s==3:
                sm = RandomOverSampler(random_state=0)
            if s==4:
                sm = BorderlineSMOTE(random_state=0) 
            if s==5:
                sm = SMOTETomek(random_state=0)
            #sm = KMeansSMOTE(random_state=0)  #X need 25 samples
            
            x_res, y_res = sm.fit_resample(x_train1, y_train1)
            
            #print(x_train)
            #print('Resampled dataset shape %s' % Counter(y_res))

        x_train_new = []
        for rec in x_res:
            x_train_new.append([rec])
            #x_train_new.append(np.reshape(rec,(-1,self.EMB_SIZE)))
        x_test_new = []
        for rec1 in x_test1:
            x_test_new.append([rec1])
                
        x_train_new = np.array(x_train_new)  
        x_test_new = np.array(x_test_new)
        #print(x_train_new)
        
        y_train_onehot_new = []
        y_test_new = [] 
        y_test_new1 = [] 
        
        for sid in y_res:
            index = int(sid)
            sidnum = [0]*n
           
            #print(index)
            sidnum[index-1] = 1
            #print(index)
            #print(sidnum)
            y_train_onehot_new.append(sidnum)  
        for sidstr in y_test1:
            index = int(sidstr)
            sidnum1 = [0]*n
            
            #print(index)
            sidnum1[index-1] = 1
            #print(sidnum)
            y_test_new.append(sidnum1) 
            y_test_new1.append([sidnum1])

        return x_train_new, y_train_onehot_new, x_test_new, y_test_new, y_test_new1

######################################################
    def build_model(self,num):
        return self.simple_model(num)

######################################################
    def simple_model(self,num):
        model = Sequential()
        model.add(Conv1D(input_shape=(1, self.EMB_SIZE), filters=128, kernel_size=16, padding='same'))
        model.add(BatchNormalization())
        model.add(LeakyReLU())
        model.add(Dropout(0.5))

        model.add(Conv1D(filters=64, kernel_size=8, padding='same'))
        model.add(BatchNormalization())
        model.add(LeakyReLU())
        model.add(Dropout(0.5))
       
        model.add(Flatten())
        model.add(Dense(128))
        model.add(BatchNormalization())
        model.add(LeakyReLU())
        if num==0:
            model.add(Dense(40))
        if num==1:
            model.add(Dense(37))
        if num==2:
            model.add(Dense(16))
        if num==3:
            model.add(Dense(84))
            
        model.add(Activation('softmax'))

        opt = Nadam(lr=0.002)
        
        model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=[auc])
        #model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
        #print(model.summary())
        return model

######################################################
    def train_model(self, model, x_train, y_train_onehot, x_test1, y_test1 ):
        model_name = 'cnn.'+self.stkid+'.model'
        input_shape = x_train[0].shape
        #reduce_lr = ReduceLROnPlateau(monitor='roc_auc_val', factor=0.8, patience=30, min_lr=0.000001, verbose=2)
        reduce_lr = ReduceLROnPlateau(monitor='roc_accuracy', factor=0.8, patience=30, min_lr=0.000001, verbose=2)
        checkpointer = ModelCheckpoint(filepath=model_name, verbose=0, save_best_only=True)
        
        train_history = model.fit(np.array(x_train),np.array(y_train_onehot), epochs = 200, batch_size = 10, verbose=2, \
                                  validation_data=(np.array(x_test1), np.array(y_test1)), callbacks=[reduce_lr, checkpointer], shuffle=True)
        '''
        train_history = model.fit(np.array(x_train),np.array(y_train_onehot), epochs = 200, batch_size = 100, verbose=0, \
                                   callbacks=[reduce_lr, checkpointer], shuffle=True)
        '''
        #show_train_history(train_history, 'accuracy', 'val_accuracy')
        #show_train_history(train_history, 'auc', 'val_auc')
        
        #show_train_history(train_history, 'loss', 'val_loss')
        return train_history

######################################################
    def do_backtest(self,num,s):

        # cut data 
        x_train, y_train_onehot, x_test1, y_test1, y_test2 = self.backtest_training_data(num,s)
        # build model 
        model = self.build_model(num)
        train_history = self.train_model(model, x_train, y_train_onehot, x_test1, y_test1 )
        
        if num==0:
            n = 40
        if num==1:
            n = 37
        if num==2:
            n = 16
        if num==3:
            n = 84
            
        count = 0
        total = 0
        #print(x_test1)
        #print(np.array(y_test2))
        #print("x : "+str(x_test1.shape)+" / y : "+str(np.array(y_test2).shape))
        for x_test, y_test in zip(x_test1, np.array(y_test2)):
            
            prediction = model.predict_classes(np.array([x_test]),verbose=0)
            #print(proba[0], proba[1])
            #print(datestr, prediction[0], y_test_onehot[0], scores[1], last_close)
            
      
            index = int(prediction)+1
            #print("prediction: "+str(index))
        
            sidnum = [0]*n
            #print(index)
            sidnum[index-1] = 1
            sidnum_x = np.array(sidnum)
            
            total += 1
            
            if (y_test[0][index-1] == sidnum_x[index-1]):
                count += 1
        if (total == 0):
            return 0, 0
        
    
        #print(total)
        #print(count)
        return count, total
        
######################################################
"""
def show_train_history(train_history, train, validation):
    plt.plot(train_history.history[train])
    #plt.plot(train_history.history[validation])
    plt.title('Train History')
    plt.ylabel(train)
    plt.xlabel('Epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()
"""
def main():
    print("start")
    stkid = 'ALL_fiducial_2' 
    TID = 'fiducial'
    DA = ["AUSAIC Data","MIT-BIH Arrhythmia Data","MIT-BIH Normal Sinus Rhythm Data","QT Data"]
    SMO = ["Original","SMOTE","SVMSMOTE","ROS","BorderlineSMOTE","SMOTETomek"]
    ADD = ["/home/keg/桌面/ecg_data/AUSAIC Data/fiducial/",
          "/home/keg/桌面/ecg_data/MIT-BIH Arrhythmia Data/fiducial/",
          "/home/keg/桌面/ecg_data/MIT-BIH Normal Sinus Rhythm Data/fiducial/",
          "/home/keg/桌面/ecg_data/QT Data/fiducial/"]
    NUMB = [0, 1, 2]
    NUMBE = [0, 1, 2, 3, 4, 5]
    info_size = 8
    onehot_size = 0
    #file = open('/home/keg/桌面/ecg_data/fiducial_accuracy-1.csv', 'w')
    #file.write('Method,DATA,Ture/Total,accuracy'+'\n')
    for num in NUMB:
        base_path = ADD[num]
        #S=0  #SMOTE /S=1  #SVMSMOTE /S=2  #ROS  /S=3  #BorderlineSMOTE  /S=4  #SMOTETomek
        for S in NUMBE:
            ecg = ecgcnn(base_path, stkid, info_size, onehot_size )
            count, total = ecg.do_backtest(num,S)
            hitrate = 0.0
            if (total > 0):
                hitrate = float(count)/total
                #print(SMO[S]+","+DA[num]+",("+str(count)+"/"+str(total)+"),"+str(hitrate))
                print(SMO[S]+","+DA[num])
                plan =SMO[S]+","+DA[num]+",("+str(count)+"/"+str(total)+"),"+str(hitrate)+'\n'
                #file = open('/home/keg/桌面/ecg_data/fiducial_accuracy.txt', 'w')
                #file.write(plan)
    #file.close()
    print("finish")
    #xt, yt = ecg.backtest_training_data()
    #print(xt)
    #print(yt[-1])
    #for stkid in stklist:    
    #    model = ecgcnn(base_path, stkid,info_size)
                      
if (__name__ == "__main__"):
    main()


/home/keg/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/keg/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/keg/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/keg/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: 

start


/home/keg/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:80: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
W0118 15:57:29.594923 140462657562368 deprecation_wrapper.py:119] From /home/keg/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



Train on 3484 samples, validate on 871 samples
Epoch 1/200
 - 36s - loss: 2.6358 - auc: 0.8540 - val_loss: 2.1097 - val_auc: 0.9601


/home/keg/anaconda3/lib/python3.6/site-packages/keras/callbacks/callbacks.py:1042: RuntimeWarning: Reduce LR on plateau conditioned on metric `roc_accuracy` which is not available. Available metrics are: val_loss,val_auc,loss,auc,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


...

Train on 2128 samples, validate on 263 samples
Epoch 1/200
 - 52s - loss: 1.3811 - auc: 0.9166 - val_loss: 1.5856 - val_auc: 0.9667


/home/keg/anaconda3/lib/python3.6/site-packages/keras/callbacks/callbacks.py:1042: RuntimeWarning: Reduce LR on plateau conditioned on metric `roc_accuracy` which is not available. Available metrics are: val_loss,val_auc,loss,auc,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


Epoch 2/200
 - 10s - loss: 0.8844 - auc: 0.9722 - val_loss: 0.4984 - val_auc: 0.9942
Epoch 3/200
 - 10s - loss: 0.7314 - auc: 0.9795 - val_loss: 0.3556 - val_auc: 0.9962
Epoch 4/200
 - 10s - loss: 0.6987 - auc: 0.9806 - val_loss: 0.2983 - val_auc: 0.9973
Epoch 5/200
 - 10s - loss: 0.6670 - auc: 0.9805 - val_loss: 0.2641 - val_auc: 0.9972
Epoch 6/200
 - 10s - loss: 0.6476 - auc: 0.9821 - val_loss: 0.2549 - val_auc: 0.9983
Epoch 7/200
 - 10s - loss: 0.6090 - auc: 0.9830 - val_loss: 0.2395 - val_auc: 0.9981
Epoch 8/200
 - 10s - loss: 0.5678 - auc: 0.9855 - val_loss: 0.2360 - val_auc: 0.9973
Epoch 9/200
 - 10s - loss: 0.5343 - auc: 0.9860 - val_loss: 0.3040 - val_auc: 0.9939
Epoch 10/200
 - 10s - loss: 0.6252 - auc: 0.9822 - val_loss: 0.2338 - val_auc: 0.9978
Epoch 11/200
 - 10s - loss: 0.5315 - auc: 0.9876 - val_loss: 0.2992 - val_auc: 0.9940
Epoch 12/200
 - 10s - loss: 0.5712 - auc: 0.9860 - val_loss: 0.2624 - val_auc: 0.9977
Epoch 13/200
 - 10s - loss: 0.5395 - auc: 0.9864 - val_loss: 0

Epoch 98/200
 - 10s - loss: 0.3469 - auc: 0.9921 - val_loss: 0.1799 - val_auc: 0.9988
Epoch 99/200
 - 10s - loss: 0.3429 - auc: 0.9927 - val_loss: 0.1914 - val_auc: 0.9987
Epoch 100/200
 - 10s - loss: 0.3258 - auc: 0.9938 - val_loss: 0.1916 - val_auc: 0.9986
Epoch 101/200
 - 10s - loss: 0.3545 - auc: 0.9922 - val_loss: 0.1549 - val_auc: 0.9994
Epoch 102/200
 - 10s - loss: 0.3272 - auc: 0.9944 - val_loss: 0.1798 - val_auc: 0.9987
Epoch 103/200
 - 10s - loss: 0.3546 - auc: 0.9916 - val_loss: 0.1606 - val_auc: 0.9989
Epoch 104/200
 - 10s - loss: 0.3557 - auc: 0.9933 - val_loss: 0.1737 - val_auc: 0.9986
Epoch 105/200
 - 10s - loss: 0.3242 - auc: 0.9930 - val_loss: 0.1637 - val_auc: 0.9988
Epoch 106/200
 - 10s - loss: 0.3163 - auc: 0.9944 - val_loss: 0.1687 - val_auc: 0.9987
Epoch 107/200
 - 10s - loss: 0.3337 - auc: 0.9928 - val_loss: 0.1955 - val_auc: 0.9984
Epoch 108/200
 - 10s - loss: 0.3304 - auc: 0.9938 - val_loss: 0.1704 - val_auc: 0.9989
Epoch 109/200
 - 10s - loss: 0.3068 - auc: 0.

Epoch 193/200
 - 10s - loss: 0.2887 - auc: 0.9954 - val_loss: 0.1441 - val_auc: 0.9993
Epoch 194/200
 - 10s - loss: 0.2671 - auc: 0.9952 - val_loss: 0.1390 - val_auc: 0.9991
Epoch 195/200
 - 10s - loss: 0.3237 - auc: 0.9924 - val_loss: 0.1531 - val_auc: 0.9990
Epoch 196/200
 - 10s - loss: 0.2737 - auc: 0.9947 - val_loss: 0.1688 - val_auc: 0.9987
Epoch 197/200
 - 10s - loss: 0.3081 - auc: 0.9947 - val_loss: 0.1505 - val_auc: 0.9991
Epoch 198/200
 - 10s - loss: 0.2633 - auc: 0.9954 - val_loss: 0.1355 - val_auc: 0.9994
Epoch 199/200
 - 10s - loss: 0.2781 - auc: 0.9956 - val_loss: 0.1413 - val_auc: 0.9989
Epoch 200/200
 - 10s - loss: 0.2391 - auc: 0.9970 - val_loss: 0.1315 - val_auc: 0.9990
SMOTETomek,MIT-BIH Normal Sinus Rhythm Data
finish
